In [ ]:
import pandas as pd 
wards = pd.read_pickle('wards.p')
print(wards.head())
print(wards.shape)

In [ ]:
census = pd.read_pickle('census.p')
print(census.head())
print(census.shape)

In [ ]:
wards_census = wards.merge(census, on='ward')
print(wards_census.head(4))
print(wards_census.shape)

In [ ]:
print(wards_census.columns)

In [ ]:
wards_census = wards.merge(census, on='ward', suffixes=('_ward', '_cen'))
print(wards_census.head())
print(wards_census.shape)

In a one-to-many relationship, every row in the left table is related to one or more rows in the right table. Within each ward, there are many businesses. We will merge the wards table with a table of licensed businesses in each ward. The business license data is stored in another table called licenses. It holds info such as the business address and ward the business is located within. The two DataFrames are related to each other by their ward column. When we merge the two tables together with the merge method, setting the 'on' attribute to the column ward, the resulting table has both local ward data and business license data. Notice that ward 1 and its alderman Joe is repeated in the resulting table because the licenses table has many businesses in the 1st ward. pandas takes care of the one-to-many relationships for us and doesn't require anything special on our end. We can use the same syntax as we did with one-to-one relationships.

In [ ]:
licenses = pd.read_pickle('licenses.p')
ward_licenses = wards.merge(licenses, on='ward', suffixes=('_ward', '_lic'))
print(ward_licenses.head())
print(ward_licenses.shape)

Sometimes we need to merge together more than just two tables to complete our analysis. We used two tables from the city of Chicago. One table contained business licenses issued by the city. The other table listed info about the local neighborhoods called wards, including the local government official's office. Now, we also have a table of businesses that have received small business grant money from Chicago. The grants are funded by taxpayer money. Therefore, it would be helpful to analyze how much grant money each business received and in what ward that business is located. We then could determine if one ward's businesses received a disproportionately large amount of grant money. To pull all of this information together, let's first connect our grants table to our licenses table. The two tables are related by their company name and location. 

If we merge the two tables only using the zip column, then the 60616 zip of Reggie's bar from the licenses table will be matched to multiple businesses in the grants table with the same zip. Our code sample prints the first few rows and some columns of the merged table. The output of the merge duplicates Reggie's bar for each matching zip in the grants table, which is not what we want. If instead, we merged on address only, there's a small risk that the address would repeat in different parts of the city. Therefore, the best option is to merge the tables using the combination of both address and zip code.

We merge the two DataFrames as shown before, except in this case, we pass a list of the column names we want to merge on to the 'on' argument. This allows us to use multiple columns in the merge. As before, the matching rows between the two DataFrames are returned with the columns from the grant table listed first. However, when we merge on two columns, in this case address and zip code, we are requiring that both the address and zip code of a row in the left table match the address and zip code of a row in the right table in order for them to be linked to each other in the merge.

In [ ]:
# grants file does not exist. This code will result in a NameError.
grants_licenses = grants.merge(licenses, on=['address', 'zip'])

We can now extend this example to a third table. First, we merge the grants table with the wards table on the ward column again, adding suffixes to the repeated column names. Note that we're using Python's backslash line continuation method to add the second merge on the next line. Python will read this as just one line of code. Without this, Python will throw a syntax error since it will parse it as two separate lines of code, so don't forget your backslash. Now our output table has information about grants, business, and wards. We can now complete our analysis.

In [ ]:
grants_licenses_ward = grants.merge(licenses, on=['address', 'zip']) \
                            .merge(wards, on='ward', suffixes=('_bus', '_ward'))

We can now sum the grants by ward and plot the results. Some wards have received more grants than others.

In [ ]:
import matplotlib.pyplot as plt
grant_licenses_ward.groupby('ward').agg('sum').plot(kind='bar', y='grant')
plt.show()